<a href="https://colab.research.google.com/github/YasinEnigma/Scripts/blob/master/graph_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/amnghd/Persian_poems_corpus

Cloning into 'Persian_poems_corpus'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 45.21 MiB | 18.24 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (148/148), done.


In [ ]:
!mv Persian_poems_corpus/original/ ./

In [ ]:
!mkdir normalized 'stop words removed'

In [ ]:
!pip install node2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
!pip install pecanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import networkx as nx

from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings("ignore")

import random
from tqdm import tqdm


# below is a context graph, that represents words that are adjencent to each other with count as weight

In [ ]:
from glob import glob # using glob to input all the poem files
from pers_alphab import pers_alphab # used to normalize the poem alphabet
import re # for quick split function
from time import time # to time the code

In [ ]:
def stop_word_importer(file_name):# importing persian stopwords
    with open(file_name, 'r', encoding="utf8") as myfile:
        stop_words = myfile.read().replace('\n', ' ').replace("\u200c","").replace("\ufeff","").replace("."," ").split(' ')# a list of stop words
    return stop_words


In [ ]:
original_path = 'original/*.txt'  # where the original (scraped) files exist
poet_files = glob(original_path)  # captures list of all the files in the folder
stop_words = stop_word_importer('stop_words.txt') # importing stop words to a list

In [ ]:
def document_traverser(original_path, normalized_path, stop_remv_path):
    # one read two writes are initiated
    with open(original_path, "r", encoding="utf8") as f,\
    open(normalized_path, "w",  encoding="utf8") as n,\
    open(stop_remv_path, "w",  encoding="utf8") as s:
        for line in f:
            normalized = pers_alphab(line) # normelizing the content
            word_list = re.split('[\t\s:]+', normalized) # tokenizing the content
            cleaned_words = [x for x in word_list if x not in stop_words] # no stop word is in it
            n.write(normalized) # writing out the normalized documents
            s.write(" ".join(cleaned_words) + '\n')  # writing out the stop word removed documents

In [ ]:
for poet in poet_files: # going over the list of all poems
    poet_name = poet[9:-4] # taking poems name out for output files
    document_traverser(poet, "normalized/" + poet_name + "_norm.txt",
                       "stop words removed/" + poet_name + "_stp_rmv.txt")

In [ ]:
!cp -r normalized/ poems/

In [ ]:
import glob

read_files = glob.glob("poems/*.txt") # reading all the files into one list

with open("poems/whole_farsi_corpus.txt", "w", encoding="utf8") as outfile:
    for f in read_files:
        with open(f, "r", encoding="utf8") as infile:
            outfile.write(infile.read())

In [ ]:
read_files

['poems/obeyd_norm.txt',
 'poems/saeb_norm.txt',
 'poems/khajoo_norm.txt',
 'poems/parvin_norm.txt',
 'poems/saadi_norm.txt',
 'poems/attar_norm.txt',
 'poems/khosro_norm.txt',
 'poems/ouhadi_norm.txt',
 'poems/bidel_norm.txt',
 'poems/shahnematollah_norm.txt',
 'poems/asad_norm.txt',
 'poems/sanaee_norm.txt',
 'poems/hatef_norm.txt',
 'poems/ghaani_norm.txt',
 'poems/moulavi_norm.txt',
 'poems/gilani_norm.txt',
 'poems/babaafzal_norm.txt',
 'poems/rahi_norm.txt',
 'poems/abusaeed_norm.txt',
 'poems/naserkhosro_norm.txt',
 'poems/orfi_norm.txt',
 'poems/khayyam_norm.txt',
 'poems/zahir_norm.txt',
 'poems/razi_norm.txt',
 'poems/shabestari_norm.txt',
 'poems/asadi_norm.txt',
 'poems/bahaee_norm.txt',
 'poems/farrokhi_norm.txt',
 'poems/hafez_norm.txt',
 'poems/helali_norm.txt',
 'poems/salman_norm.txt',
 'poems/feyz_norm.txt',
 'poems/seyf_norm.txt',
 'poems/nezari_norm.txt',
 'poems/vahshi_norm.txt',
 'poems/shahriar_norm.txt',
 'poems/kamal_norm.txt',
 'poems/iqbal_norm.txt',
 'poems/

In [ ]:
import re  # to perform a couple of simple regexes
from gensim.models import Word2Vec  # to perform word2vec modelling
from time import time  # to monitor the run time

tic = time()  # starting time
poems_path = r"poems/"
poet_name = "whole_farsi_corpus.txt"
pattern = re.compile(r"[\n\t]+")  # splitting on tabs or lines (multiples)

with open(poems_path + poet_name, 'r', encoding="utf8") as file:
    raw_string = file.read()  # reading the poet document into a string
    raw_string = pattern.sub(raw_string, "\n")  # replacing all\n\t with\n
    raw_string = raw_string.replace("[", "").replace("]", "")  # removes [ , ]
    unique_chars = set(raw_string) - set("\n\s\t ")  # generate unique chars
    mesras = raw_string.split("\n")  # getting a list of mesras(clauses)
    words = re.split(r"[\n\s\t ]+", raw_string)  # getting a list of words
    unique_words = set(words)  # unique words


list_of_sentences = []  # developing a list of sentences to be used for Gsnim
sentence_max_len = 0  # maximum length of the sentences initiaton
sentence_min_len = 100  # minimum length of the sentences initiaton
for mesra in mesras:
    if mesra.strip():  # not an empty line is faced
        mesra_list = [word for word in mesra.strip().split(" ") if word]
        list_of_sentences.append(mesra_list)
        if len(mesra_list) > sentence_max_len:
            sentence_max_len = len(mesra_list)  # for troubleshooting
        if len(mesra_list) < sentence_min_len:
            sentence_min_len = len(mesra_list)  # for troubleshooting

cache = {}  # to hold informations to be transferred
cache["num_of_uwords"] = len(unique_words)  # caching number of unique words
cache["num_of_chars"] = len(unique_chars)  # cachine number of characarcters
cache["num_of_words"] = len(words)  # total number of words
cache["unique_words"] = unique_words  # unique words to be used
cache["sentence_min_max"] = (sentence_min_len, sentence_max_len)  # range
cache["num_mesra"] = len(list_of_sentences)  # number of mesras

print("There are {} number of mesras in this document.".format(cache["num_mesra"]))
print("Mesra length range from {} words to {} words.".format(sentence_min_len, sentence_max_len))
print("There are {} unique words in this document.".format(cache["num_of_uwords"]))
print("There are {} unique characters in this document.".format(cache["num_of_chars"]))
print("In total there are {} words in this document.".format(cache["num_of_words"]))

# following hyper parameters are checked to finalize propoer hyperparameters
# size in [10, 100, 1000]
# window in [5, 40, 320]
# min_count in [1, 10, 100]
# developing the word2vec funtion


There are 1216286 number of mesras in this document.
Mesra length range from 1 words to 18 words.
There are 148558 unique words in this document.
There are 36 unique characters in this document.
In total there are 8102119 words in this document.


In [ ]:
data = list_of_sentences.copy()

In [ ]:
G =  nx.DiGraph()
for s in data:
    G.add_nodes_from(s)
    for i,w in enumerate(s[1:]):
        if (s[i],w) not in G.edges:
            G.add_edge(s[i],w,weight=1)
        else:
            G[s[i]][w]["weight"]+=1

In [ ]:
len(list(G.nodes()))

148558

In [ ]:
vocabulary = ["NA"] + list(G.nodes)
vocabulary_lookup = {token: idx for idx, token in enumerate(vocabulary)}

In [ ]:
print("Total number of graph nodes:", G.number_of_nodes())
print("Total number of graph edges:", G.number_of_edges())

Total number of graph nodes: 148558
Total number of graph edges: 2171499


In [ ]:
print('Density is: ', (G.number_of_edges()/(G.number_of_nodes()**2)))

Density is:  9.839375746861411e-05


In [ ]:
degrees = []
for node in G.nodes:
    degrees.append(G.degree[node])

print("Average node degree:", round(sum(degrees) / len(degrees), 2))


Average node degree: 29.23


In [ ]:
nx.write_edgelist(G, "graph.edgelist", data=["weight"], encoding='utf-8')

In [ ]:
from pecanpy import pecanpy# load graph object using SparseOTF mode
g = pecanpy.SparseOTF(p=0.8, q=0.1, workers=4, verbose=False)
g.read_edg("graph.edgelist", weighted=True, directed=True, delimiter=' ')

In [ ]:
# generate random walks
walks = g.simulate_walks(num_walks=30, walk_length=50)

In [ ]:
size = 100
window = 5
min_count =100
w2v_model = Word2Vec(
                walks,
                sg=1,
                vector_size=size,
                window=window,
                min_count=min_count,
                workers=4)
w2v_model.train(walks, total_examples=len(walks), epochs=10)

(1267160180, 1416409090)

In [ ]:
# saving the model vector into a txt file
w2v_model.wv.save_word2vec_format(pos1 = u'زلیخا'  # reason
pos2 = u'مجنون'
neg = u'یوسف'
result = w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg])
print(neg + u" به " + pos1 + " شبیه " + pos2 + " است:\n{}".format(*result[0]))
, binary=False)

SyntaxError: ignored

In [ ]:
wv_from_text = KeyedVectors.load_word2vec_format(datapath('word2vec_pre_kv_c'), binary=False)

In [ ]:
w1 = u'خدا'
print("Few most similar words to \"" + w1 + "\" are:\n" + "\n".join(str(word) +":" + str(ratio) for word, ratio in w2v_model.wv.most_similar(positive=w1, topn=3)))

Few most similar words to "خدا" are:
پندری:0.794999897480011
جوزایه:0.6950055956840515
پیدایه:0.6883607506752014


In [ ]:
w1 = u'عشق'
print("Few most similar words to \"" + w1 + "\" are:\n" + "\n".join(str(word) +":" + str(ratio) for word, ratio in w2v_model.wv.most_similar(positive=w1, topn=3)))

Few most similar words to "عشق" are:
داغداری:0.7580649852752686
عاشقی:0.6385080814361572
عشقت:0.6222206354141235


In [ ]:
w1 = u'حکایت'
print("Few most similar words to \"" + w1 + "\" are:\n" + "\n".join(str(word) +":" + str(ratio) for word, ratio in w2v_model.wv.most_similar(positive=w1, topn=3)))

Few most similar words to "حکایت" are:
قصه:0.734473466873169
جویانم:0.6380172967910767
حدیث:0.6314422488212585


In [ ]:
w1 = u'شهر'
print("Few most similar words to \"" + w1 + "\" are:\n" + "\n".join(str(word) +":" + str(ratio) for word, ratio in w2v_model.wv.most_similar(positive=w1, topn=3)))

Few most similar words to "شهر" are:
خطه:0.699218213558197
کجاران:0.674374520778656
آنجایند:0.642934262752533


In [ ]:
w1 = u'رستم'
print("Few most similar words to \"" + w1 + "\" are:\n" + "\n".join(str(word) +":" + str(ratio) for word, ratio in w2v_model.wv.most_similar(positive=w1, topn=5)))

Few most similar words to "رستم" are:
زابلیست:0.8165891766548157
زال:0.7275353670120239
وطابران:0.7157247066497803
دستان:0.7041464447975159
نریمان:0.7017806768417358


In [ ]:
pos1 = u'زن'  # reason
pos2 = u'آدم'
neg = u'مرد'
result = w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg])
print(neg + u" به " + pos1 + " شبیه " + pos2 + " است:\n{}".format(*result[0]))


In [ ]:
# let's perform an anlogy test between ismaeil, ibrahim, hossein, ali
pos1 = u'چشم'  # reason
pos2 = u'شنیده'
neg = u'گوش'
result = w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg])
print(neg + u" به " + pos1 + " شبیه " + pos2 + " است:\n{}".format(*result[0]))


# this example is very important

pos1 = u'رستم'  # reason
pos2 = u'پدر'
neg = u'زال'
result = w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg])
print(neg + u" به " + pos1 + " شبیه " + pos2 + " است:\n{}".format(*result[0]))


pos1 = u'زلیخا'  # reason
pos2 = u'مجنون'
neg = u'یوسف'
result = w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg])
print(neg + u" به " + pos1 + " شبیه " + pos2 + " است:\n{}".format(*result[0]))



In [ ]:

pos1 = u'راست'  # reason
pos2 = u'بالا'
neg = u'چپ'
result = w2v_model.wv.most_similar(positive=[pos1, pos2], negative=[neg])
print(neg + u" به " + pos1 + " شبیه " + pos2 + " است:\n{}".format(*result[0]))

